In [253]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [254]:
data = pd.read_csv("LoanStats_securev1_2019Q4.csv", low_memory = False)

In [255]:
finished_loans = data.loc[(data['loan_status'] == 'Default') | (data['loan_status'] == 'Charged Off') |
                          (data['loan_status'] == 'Fully Paid')]
finished_loans.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
45,164190449,NaN,15000.0,15000.0,15000.0,60 months,23.05%,423.29,D,D3,Social Worker,7 years,MORTGAGE,63800.0,Source Verified,Dec-2019,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,credit_card,Credit card refinancing,666xx,KS,18.91,2.0,Aug-2004,660.0,664.0,0.0,18.0,NaN,5.0,0.0,14275.0,69.6%,54.0,w,0.0,0.0,15134.460833,15134.46,15000.0,134.46,0.0,0.0,0.0,Feb-2020,163.27,NaN,Jan-2020,669.0,665.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,208146.0,0.0,2.0,0.0,1.0,15.0,193871.0,84.0,0.0,0.0,11757.0,78.0,20500.0,3.0,4.0,4.0,2.0,41629.0,2194.0,84.9,0.0,0.0,184.0,173.0,31.0,10.0,3.0,41.0,20.0,4.0,18.0,0.0,2.0,3.0,2.0,6.0,38.0,3.0,13.0,3.0,5.0,0.0,0.0,0.0,1.0,94.0,100.0,0.0,0.0,175764.0,208146.0,14500.0,155264.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
118,163396715,NaN,12000.0,12000.0,12000.0,36 months,7.56%,373.61,A,A3,Fleet Service Agent,10+ years,RENT,80000.0,Not Verified,Dec-2019,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,other,Other,852xx,AZ,9.18,0.0,Feb-2009,725.0,729.0,1.0,NaN,NaN,23.0,0.0,9057.0,13.1%,38.0,f,0.0,0.0,12007.812000,12007.81,12000.0,7.81,0.0,0.0,0.0,Jan-2020,12015.37,NaN,Jan-2020,729.0,725.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,39951.0,4.0,9.0,0.0,0.0,28.0,30006.0,88.0,5.0,5.0,4986.0,39.0,69000.0,1.0,4.0,7.0,5.0,1737.0,36060.0,14.5,0.0,0.0,106.0,130.0,0.0,0.0,0.0,2.0,NaN,2.0,NaN,0.0,4.0,6.0,7.0,13.0,15.0,13.0,22.0,6.0,23.0,0.0,0.0,0.0,5.0,100.0,14.3,0.0,0.0,103877.0,39951.0,42200.0,33989.0,NaN,NaN,NaN,NaN,NaN,NaN,N

In [256]:
cleaned_data = finished_loans


def emp_length_to_num(inp):
    if inp == "10+ years":
        return 10
    elif inp == "< 1 year":
        return 1
    elif inp == '':
        return 0
    else:
        return int(inp[0])

emp_length = cleaned_data["emp_length"].fillna('').apply(emp_length_to_num)
del cleaned_data['emp_length']
cleaned_data.insert(2, "emp_length", emp_length, True)


grade = cleaned_data['grade'].replace(["A", "B", "C", "D", "E", "F", "G"], [0, 1, 2, 3, 4, 5, 6])
del cleaned_data['grade']
cleaned_data.insert(2, "grade", grade, True)


def sub_grade_to_num(inp):
    return int(inp[1])

sub_grade = cleaned_data["sub_grade"].fillna('').apply(sub_grade_to_num)
del cleaned_data['sub_grade']
cleaned_data.insert(2, "sub_grade", sub_grade, True)


def int_rate_to_num(inp):
    return float(inp[:-1])

int_rate = cleaned_data["int_rate"].fillna('').apply(int_rate_to_num)
del cleaned_data['int_rate']
cleaned_data.insert(2, "int_rate", int_rate, True)


#for training use issue_d, for inferencing use today
days_since_earliest_cr_line = pd.to_datetime(cleaned_data["issue_d"], format = '%b-%Y') - pd.to_datetime(cleaned_data["earliest_cr_line"], format = '%b-%Y')
cleaned_data.insert(2, "days_since_earliest_cr_line", days_since_earliest_cr_line, True)


#for training use issue_d, for inferencing use today
days_since_sec_app_earliest_cr_line = pd.to_datetime(cleaned_data["issue_d"], format = '%b-%Y') - pd.to_datetime(cleaned_data["sec_app_earliest_cr_line"], format = '%b-%Y')
cleaned_data.insert(2, "days_since_sec_app_earliest_cr_line", days_since_sec_app_earliest_cr_line, True)


cleaned_data.head(10)

,id,member_id,days_since_sec_app_earliest_cr_line,days_since_earliest_cr_line,int_rate,sub_grade,grade,emp_length,loan_amnt,funded_amnt,funded_amnt_inv,term,installment,emp_title,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
45,164190449,NaN,NaT,5600 days,23.05,3,3,7,15000.0,15000.0,15000.0,60 months,423.29,Social Worker,MORTGAGE,63800.0,Source Verified,Dec-2019,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,credit_card,Credit card refinancing,666xx,KS,18.91,2.0,Aug-2004,660.0,664.0,0.0,18.0,NaN,5.0,0.0,14275.0,69.6%,54.0,w,0.0,0.0,15134.460833,15134.46,15000.0,134.46,0.0,0.0,0.0,Feb-2020,163.27,NaN,Jan-2020,669.0,665.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,208146.0,0.0,2.0,0.0,1.0,15.0,193871.0,84.0,0.0,0.0,11757.0,78.0,20500.0,3.0,4.0,4.0,2.0,41629.0,2194.0,84.9,0.0,0.0,184.0,173.0,31.0,10.0,3.0,41.0,20.0,4.0,18.0,0.0,2.0,3.0,2.0,6.0,38.0,3.0,13.0,3.0,5.0,0.0,0.0,0.0,1.0,94.0,100.0,0.0,0.0,175764.0,208146.0,14500.0,155264.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
118,163396715,NaN,NaT,3955 days,7.56,3,0,10,12000.0,12000.0,12000.0,36 months,373.61,Fleet Service Agent,RENT,80000.0,Not Verified,Dec-2019,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,other,Other,852xx,AZ,9.18,0.0,Feb-2009,725.0,729.0,1.0,NaN,NaN,23.0,0.0,9057.0,13.1%,38.0,f,0.0,0.0,12007.812000,12007.81,12000.0,7.81,0.0,0.0,0.0,Jan-2020,12015.37,NaN,Jan-2020,729.0,725.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,39951.0,4.0,9.0,0.0,0.0,28.0,30006.0,88.0,5.0,5.0,4986.0,39.0,69000.0,1.0,4.0,7.0,5.0,1737.0,36060.0,14.5,0.0,0.0,106.0,130.0,0.0,0.0,0.0,2.0,NaN,2.0,NaN,0.0,4.0,6.0,7.0,13.0,15.0,13.0,22.0,6.0,23.0,0.0,0.0,0.0,5.0,10

In [257]:
cleaned_data = finished_loans

In [258]:
primary_dependent_variables = finished_loans[['loan_amnt', 'funded_amnt', 'term', 'int_rate', 'installment', 'grade',
                                    'sub_grade', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
                                    'purpose', 'zip_code', 'addr_state', 'emp_title', 'acc_now_delinq',
                                    'acc_open_past_24mths', 'bc_open_to_buy', 'percent_bc_gt_75', 'bc_util',
                                    'dti', 'delinq_2yrs', 'delinq_amnt', 'days_since_earliest_cr_line', 'fico_range_low',
                                    'fico_range_high', 'inq_last_6mths', 'mths_since_last_delinq',
                                    'mths_since_last_record', 'mths_since_recent_inq', 'mths_since_recent_revol_delinq',
                                    'mths_since_recent_bc', 'mort_acc', 'open_acc', 'pub_rec', 'total_bal_ex_mort',
                                    'total_bc_limit', 'total_acc', 'total_il_high_credit_limit', 'num_rev_accts', 
                                    'mths_since_recent_bc_dlq', 'pub_rec_bankruptcies', 'chargeoff_within_12_mths',
                                    'collections_12_mths_ex_med', 'tax_liens', 'mths_since_last_major_derog',
                                    'num_sats', 'num_tl_op_past_12m', 'tot_cur_bal', 'avg_cur_bal', 'num_bc_tl',
                                    'num_actv_bc_tl', 'num_bc_sats', 'pct_tl_nvr_dlq', 'num_tl_90g_dpd_24m',
                                    'num_tl_30dpd', 'num_tl_120dpd_2m', 'num_il_tl', 'mo_sin_old_il_acct',
                                    'num_actv_rev_tl', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op',
                                    'total_rev_hi_lim', 'num_rev_tl_bal_gt_0', 'num_op_rev_tl', 'tot_coll_amt',
                                    'application_type', 'annual_inc_joint', 'dti_joint',
                                    'verification_status_joint', 'open_acc_6m', 'open_il_12m', 'open_il_24m',
                                    'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m',
                                    'max_bal_bc', 'all_util', 'inq_fi', 'total_cu_tl', 'inq_last_12m',
                                    'sec_app_fico_range_low', 'sec_app_fico_range_high', 'days_since_sec_app_earliest_cr_line',
                                    'sec_app_inq_last_6mths', 'sec_app_mort_acc', 'sec_app_open_acc',
                                    'sec_app_revol_util', 'sec_app_num_rev_accts', 'sec_app_chargeoff_within_12_mths',
                                    'sec_app_collections_12_mths_ex_med', 'sec_app_mths_since_last_major_derog',
                                    'revol_bal_joint', 'open_act_il']]

#desc has count 0
#can't find expDefaultRate, serviceFeeRate, acceptD, expD, listD, creditPullD, reviewStatusD, reviewStatus, investorCount, ilsExpD, initialListStatus, revolBal, revolUtil, numAcctsEver120Ppd, moSinRcntTl, totHiCredLim, mtgPayment, housingPayment, disbursementMethod, secAppOpenActIl

#is int_rate just a function of grade/sub_grade?
primary_dependent_variables.head()

,loan_amnt,funded_amnt,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,purpose,zip_code,addr_state,emp_title,acc_now_delinq,acc_open_past_24mths,bc_open_to_buy,percent_bc_gt_75,bc_util,dti,delinq_2yrs,delinq_amnt,days_since_earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,mths_since_recent_inq,mths_since_recent_revol_delinq,mths_since_recent_bc,mort_acc,open_acc,pub_rec,total_bal_ex_mort,total_bc_limit,total_acc,total_il_high_credit_limit,num_rev_accts,mths_since_recent_bc_dlq,pub_rec_bankruptcies,chargeoff_within_12_mths,collections_12_mths_ex_med,tax_liens,mths_since_last_major_derog,num_sats,num_tl_op_past_12m,tot_cur_bal,avg_cur_bal,num_bc_tl,num_actv_bc_tl,num_bc_sats,pct_tl_nvr_dlq,num_tl_90g_dpd_24m,num_tl_30dpd,num_tl_120dpd_2m,num_il_tl,mo_sin_old_il_acct,num_actv_rev_tl,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,total_rev_hi_lim,num_rev_tl_bal_gt_0,num_op_rev_tl,tot_coll_amt,application_type,annual_inc_joint,dti_joint,verification_status_joint,open_acc_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,inq_fi,total_cu_tl,inq_last_12m,sec_app_fico_range_low,sec_app_fico_range_high,days_since_sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,revol_bal_joint,open_act_il
45,15000.0,15000.0,60 months,23.05,423.29,3,3,7,MORTGAGE,63800.0,Source Verified,credit_card,666xx,KS,Social Worker,0.0,2.0,2194.0,100.0,84.9,18.91,2.0,0.0,5600 days,660.0,664.0,0.0,18.0,NaN,4.0,18.0,41.0,3.0,5.0,0.0,208146.0,14500.0,54.0,155264.0,13.0,20.0,0.0,0.0,0.0,0.0,NaN,5.0,1.0,208146.0,41629.0,6.0,2.0,2.0,94.0,0.0,0.0,0.0,38.0,184.0,3.0,173.0,31.0,20500.0,3.0,3.0,0.0,Individual,NaN,NaN,NaN,0.0,0.0,1.0,15.0,193871.0,84.0,0.0,0.0,11757.0,78.0,3.0,4.0,4.0,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
118,12000.0,12000.0,36 months,7.56,373.61,0,3,10,RENT,80000.0,Not Verified,other,852xx,AZ,Fleet Service Agent,0.0,5.0,36060.0,14.3,14.5,9.18,0.0,0.0,3955 days,725.0,729.0,1.0,NaN,NaN,2.0,NaN,2.0,0.0,23.0,0.0,39951.0,42200.0,38.0,33989.0,22.0,NaN,0.0,0.0,0.0,0.0,NaN,23.0,5.0,39951.0,1737.0,13.0,4.0,7.0,100.0,0.0,0.0,0.0,15.0,106.0,6.0,130.0,0.0,69000.0,6.0,13.0,0.0,Individual,NaN,NaN,NaN,4.0,0.0,0.0,28.0,30006.0,88.0,5.0,5.0,4986.0,39.0,1.0,4.0,7.0,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0
129,18825.0,18825.0,36 months,6.46,576.63,0,1,1,OWN,95000.0,Source Verified,other,112xx,NY,Hardware Software Technician,0.0,4.0,29534.0,0.0,18.9,11.89,0.0,0.0,2678 days,745.0,749.0,0.0,NaN,NaN,12.0,NaN,39.0,0.0,8.0,0.0,25856.0,36400.0,16.0,21000.0,10.0,NaN,0.0,0.0,0.0,0.0,NaN,8.0,2.0,25856.0,3232.0,8.0,3.0,4.0,100.0,0.0,0.0,0.0,5.0,76.0,5.0,88.0,9.0,61700.0,5.0,6.0,0.0,Individual,NaN,NaN,NaN,0.0,1.0,2.0,12.0,14616.0,70.0,1.0,1.0,5637.0,27.0,2.0,1.0,1.0,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
155,9075.0,9075.0,36 months,6.46,277.98,0,1,5,MORTGAGE,94000.0,Not Verified,home_improvement,023xx,MA,Director of Operations,0.0,8.0,25977.0,0.0,17.0,16.76,0.0,0.0,5416 days,760.0,764.0,0.0,60.0,NaN,9.0,60.0,47.0,1.0,20.0,0.0,53321.0,31300.0,49.0,70551.0,8.0,60.0,0.0,0.0,0.0,0.0,60.0,20.0,3.0,468898.0,23445.0,5.0,4.0,4.0,85.7,0.0,0.0,0.0,40.0,178.0,4.0,169.0,12.0,40500.0,4.0,6.0,79.0,Individual,NaN,NaN,NaN,0.0,2.0,7.0,9.0,47998.0,68.0,1.0,1.0,4278.0,48.0,4.0,5.0,7.0,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0
173,21275.0,21275.0,36 months,11.02,696.72,1,2,4,RENT,80000.0,Source Verified,debt_consolidation,110xx,NY,Coordinator,0.0,7.0,3880.0,0.0,0.5,11.90,1.0,0.0,4748 days,665.0,669.0,0.0,17.0,NaN,11.0,30.0,8.0,0.0,7.0,0.0,10014.0,3900.0,16.0,19030.0,11.0,30.0,0.0,0.0,0.0,0.0,17.0,7.0,4.0,10014.0,1431.0,8.0,1.0,4.0,76.9,1.0,0.0,0.0,5.0,156.0,2.0,150.0,8.0,4700.0,2.0,5.0,0.0,Individual,NaN,NaN,NaN,0.0,1.0,2

In [259]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

primary_dependent_variables.select_dtypes(exclude=numerics)

#one hot encode term, mortgage, verification_status
#mean encode purpose, zip_code, addr_state, empt_title, application_type, verification_status_joint
#take into account grade and subgrade during mean encoding?

,term,home_ownership,verification_status,purpose,zip_code,addr_state,emp_title,days_since_earliest_cr_line,application_type,verification_status_joint,days_since_sec_app_earliest_cr_line
45,60 months,MORTGAGE,Source Verified,credit_card,666xx,KS,Social Worker,5600 days,Individual,NaN,NaT
118,36 months,RENT,Not Verified,other,852xx,AZ,Fleet Service Agent,3955 days,Individual,NaN,NaT
129,36 months,OWN,Source Verified,other,112xx,NY,Hardware Software Technician,2678 days,Individual,NaN,NaT
155,36 months,MORTGAGE,Not Verified,home_improvement,023xx,MA,Director of Operations,5416 days,Individual,NaN,NaT
173,36 months,RENT,Source Verified,debt_consolidation,110xx,NY,Coordinator,4748 days,Individual,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...
128135,36 months,RENT,Source Verified,debt_consolidation,274xx,NC,Ticket Office Manager,3348 days,Individual,NaN,NaT
128195,60 months,OWN,Source Verified,home_improvement,325xx,FL,Avionics tech,10288 days,Joint App,Source Verified,5721 days
128209,36 months,RENT,Source Verified,other,460xx,IN,Finance manager,3105 days,Individual,NaN,NaT
128251,36 months,MORTGAGE,Source Verified,debt_consolidation,112xx,NY,Bank Clerk,10775 days,Individual,NaN,NaT
